In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import re
import random
import numpy as np
import torch
from transformers import AutoTokenizer

In [7]:
path = '/content/drive/MyDrive/생성데이터/'

In [6]:
# target = '9'

In [7]:
# to_unfair_file = pd.read_csv(path + 'fair_article_' + target + '_first_preprocessing.csv')

In [8]:
# to_unfair_file['sentence'][1]

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

configuration_exaone.py:   0%|          | 0.00/9.95k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py:   0%|          | 0.00/63.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

In [172]:
def generate_response(sentence):
  prompt = prompt = f"""
  원본 문장:{sentence}
원본 문장을 불법적인 내용으로 수정합니다.
원본 문장의 형식과 내용은 반드시 지키세요.
원본 문장과 비슷한 길이로 생성합니다.
생성 문장 안에 '~~'을 쓰지 마세요.
문장을 생성할 때 불법적인 내용을 문맥과 맥락상 삽입하고, 숫자값은 특정 당사자에게 불리한 비율로 조정합니다.
'불법' 단어를 사용하지 않고 문맥상 불법적인 내용으로 수정하며, 원본 문장의 형식을 필수적으로 유지해야 합니다. 규칙을 반드시 지켜주세요.
원본 문장을 무조건 반영하고, 중간이나 끝 부분만 불법적인 요소가 들어있게 생성합니다.
상황에 따른 결과를 쓰지마세요. 지시하는 문장으로 생성하세요.
문장 종결을 잘 보세요. 단어형이면 단어형으로, 문장형이면 문장형으로 딱딱 맞춰서 쓰세요. 불법적인 요소를 반드시 넣으세요.
관련 법이 나오면, 그 법을 모두 반영하세요.
제발 시행 결과는 쓰지마라.
원본 문장의 내용을 처음부터 끝까지 반영하세요.
해설 적지 마세요.
행위의 결과 쓰지마세요.
예시 들지 마세요.
사후처리 결과 쓰지마세요.
불법적인 계약서 내용으로 바꾸세요.
"""

  messages = [
    {"role": "system",
     "content": "You are Meticulous Rephraser assistant. You review the sentence and ensure that the format is not violated."},
    {"role": "user", "content": prompt}
  ]
  input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
  )

  output = model.generate(
    input_ids.to("cuda"),
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=300,
    do_sample=True,
    temperature=0.5,
    top_k=30,
    top_p=0.97
)
  generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # 프롬프트 길이를 기준으로 생성된 부분만 추출
  prompt_length = input_ids.shape[1]  # 프롬프트 길이
  generated_text_only = generated_text[len(tokenizer.decode(input_ids[0], skip_special_tokens=True)):]

  return generated_text_only.strip()  # 양 끝 공백 제거

In [179]:
sentence = """
 4. 이 계약과 관련하여 인지한 쌍방의 영업비밀이나 정보를 부당하게 이용하는 행위
"""

In [178]:
result= generate_response(sentence)
print(result)

3. 특정 개인에게 강요하여 부당하게 상품권 또는 고가의 물품을 구매하도록 강요하는 행위, 이는 **제3자 간의 불법 계약**을 형성하며, 관련 **소비자기본법 제14조 위반**으로 처벌 대상이 됨을 명시한다.


In [11]:
def random_process_response(sentence):

    # 기준 숫자를 정규표현식으로 추출 (예: '2.')
    match = re.search(r'(\d+)\.', sentence)
    if not match:
        return generate_response(sentence)

    criterion = match.group(0)  # '2.' 형태의 기준 숫자
    criterion_number = match.group(1)  # '2' 형태의 기준 숫자

    # 문장을 기준 숫자를 기준으로 분리 (기준 숫자 전과 후)
    parts = re.split(r'(\d+\.)', sentence)  # 기준 숫자 앞뒤로 분리

    # parts[1:]을 통해 분리된 문장 부분을 사용
    before = parts[0].strip()  # 기준 숫자 전 부분
    after = ''.join(parts[1:]).strip()  # 기준 숫자 후 부분 (여러 부분이 있을 수 있음)

    # 랜덤 값 생성 (0, 1)
    random_value = random.randint(0, 1)

    # 각 경우에 따라 generate_response 적용
    if random_value == 0:
        before = generate_response(before)
    elif random_value == 1:
        after = generate_response(after)

    # 결과 합치기
    return f"{before} {criterion} {after}"

In [ ]:
# 원형 숫자 변환 함수
def get_circle_number(counter):
    return chr(9312 + counter)

# 숫자와 원형 숫자 매핑
def convert_numbers_to_circle(text):
    text = text.replace('**', '').replace('**문장:**', '').replace('\n', '').replace('\r', '')

    # 첫 3글자 내에 숫자가 있을 경우, 이를 원형 숫자로 변환
    match = re.match(r'^[0-9]{1,3}', text)  # 첫 3글자만 검사

    if match:
        # 숫자 부분 추출
        num_str = match.group()

        # 숫자 부분을 원형 숫자로 변환
        converted_num = ''.join([get_circle_number(int(digit)) for digit in num_str])

        # 변환된 숫자와 나머지 텍스트 결합
        return text.replace(num_str, converted_num, 1)

    return text

In [ ]:
# sentence = to_unfair_file['sentence'][2]  # 문장
# result_text = random_process_response(sentence)
# gen_unfair_sentence = convert_numbers_to_circle(result_text)
# print(gen_unfair_sentence)

In [ ]:
def process_multiple_sentences(to_unfair_file):
    results = []

    # sentences에서 하나씩 처리 (to_unfair_file['sentence'])
    for sentence in to_unfair_file['sentence']:
        # random_process_response 함수에서 결과 생성
        result_text = random_process_response(sentence)
        # 변환된 결과 저장
        results.append(result_text)

    # 결과를 데이터프레임으로 반환
    return pd.DataFrame(results, columns=['sentence'])

In [ ]:
def clean_sentences(temp, to_unfair_file) -> pd.DataFrame:

    temp = temp.copy()  # 원본 데이터프레임을 변경하지 않도록 복사

    for idx in temp.index:
        sentence = temp.at[idx, 'sentence']

        # 맨 앞 글자 수정
        if sentence[0] != to_unfair_file.at[idx, 'sentence'][0]:
            sentence = to_unfair_file.at[idx, 'sentence'][0] + sentence[1:]

        # '**' 제거
        sentence = sentence.replace('**', '')
        sentence = re.sub(r'\*\*.*?\*\*', '', sentence)
        sentence = re.sub(r'\*~:', '', sentence)
        sentence = sentence.replace('\n', ' ')
        sentence = sentence.replace('문장:', ' ')
        sentence = sentence.replace('특정 조건 하에:', ' ')
        sentence = re.sub(r'\*\s*\d+\s*', '', sentence)
        # '2. 2.' -> '2.'
        sentence = re.sub(r'(\d+\.)\s*\1', r'\1', sentence)
        sentence = re.sub(r'결과:.*', '', sentence)
        sentence = sentence.strip()
        temp.at[idx, 'sentence'] = sentence

    return temp


In [ ]:
target = '22'
to_unfair_file_22 = pd.read_csv('/content/drive/MyDrive/생성데이터/fair_22_article.csv')
temp = pd.read_csv('/content/drive/MyDrive/생성데이터/unfair_22_article.csv')

unfair_article = process_multiple_sentences(to_unfair_file_22)
temp_test = process_multiple_sentences(temp)
file_name = f'unfair_{target}.csv'
unfair_article.to_csv(file_name, encoding='utf-8-sig', index=False)

updated_unfair_article = clean_sentences(unfair_article)
updated_unfair_article.to_csv('unfair_22_article.csv', encoding='utf-8-sig', index=False)

updated_unfair_article = clean_sentences(temp_test)
updated_unfair_article.to_csv('temp_test.csv', encoding='utf-8-sig', index=False)

TypeError: clean_sentences() missing 1 required positional argument: 'to_unfair_file'